floyd run --gpu --mode jupyter --data itimmis/datasets/steering_new:steering_new

In [ ]:
# For deep learning
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras import backend as K
from keras.utils import np_utils
import tensorflow as tf

# Synthetic data generation (image augmentation)
from keras.preprocessing.image import ImageDataGenerator

# For image manipulation
from sklearn.feature_extraction import image
import scipy.misc

# For math
import numpy as np
from numpy import random
import math

# For file manipulation
import os
import glob
import pickle

# For graphing
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

# For reading images
from matplotlib.image import imread

# For data manipulation
from sklearn.model_selection import train_test_split

%matplotlib inline

In [ ]:
def save_training_history(history1, history2):

    save_data = (history1, history2)
    pickle_out = open('history.pickle', 'wb')
    pickle.dump(save_data, pickle_out)
    pickle_out.close()
    
    return

In [ ]:
def load_training_history():

    pickle_in = open('history.pickle', 'rb')        
    (history1, history2) = pickle.load(pickle_in)
    
    return (history1, history2)

In [ ]:
def generator_OLD(data_dict, batch_size):
    
    # This function generates each mini-batch one at a time
    
    # Create empty arrays to contain batch of features and labels
    batch_features = np.zeros((batch_size, data_dict["x_train"][1].shape[0], data_dict["x_train"][1].shape[1], data_dict["x_train"][1].shape[2]))
    batch_labels = np.zeros((batch_size, 1))
   
    while True:
        for i in range(batch_size):
            # choose random index in features
            index = random.choice(len(data_dict["x_train"]),1)
            batch_features[i] = data_dict["x_train"][index]
            batch_labels[i] = data_dict["y_train"][index]
        yield batch_features, batch_labels

In [ ]:
def generator(data_dict, batch_size):
    
    # This function generates each mini-batch one at a time
    
    # Create empty arrays to contain batch of features and labels
    batch_features = np.zeros((batch_size, 640, 480, 3))
    batch_labels = np.zeros((batch_size, 1))
   
    while True:
        for i in range(0, batch_size, 2):
            # choose random index in features
            index = random.choice(len(data_dict["x_train"]),1)
            filename = data_dict["x_train"][index]
            
            img = imread(filename)            
            radian = filename.split('_')[-1].replace('.jpg','') # Remove the idx from filepath

            # Image
            batch_features[i] = img
            batch_labels[i] = radians_to_degrees(float(radian))

            # Image Flipped
            batch_features[i + 1] = np.fliplr(img)
            batch_labels[i + 1] = radians_to_degrees(float(radian)*-1)

        batch_features /= 255.
        yield batch_features, batch_labels

In [ ]:
def radians_to_degrees(x):
    deg = math.degrees(x + math.pi)
    return deg

In [ ]:
def degrees_to_radians(x):
    rad = math.radians(x) - math.pi
    return rad

In [ ]:
def load_data_OLD():
    
    images = []
    labels = []
    
    cloud = True
    
    folder_names = ["campus_run1", "campus_run2", "run1", "run2", "run3", "second_spot1", "second_spot2"]
    
    if cloud == True:
        path = "/steering_new/"
    else: 
        path = "/Users/itimmis/Desktop/ACTor/Data/SteeringNew/"
    
    # Loop folders
    for folder in folder_names:
        
        # open folder
        os.chdir(path + folder)
        
        # Loop files
        for file in glob.glob("*.jpg"):

            # Load image
            im = imread(file)
            images.append(im)
            images.append(np.fliplr(im))

            # Extract label
            value = file.split('_')[-1].replace('.jpg','') # Remove the idx from filepath
            labels.append(radians_to_degrees(float(value)))
            labels.append(radians_to_degrees(float(value)*-1))
        
    # Convert to numpy arrays
    x = np.array(images)
    y = np.array(labels)
    
    # Extract Test Set
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1)

    # Extract Validation Set
    x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.25, random_state=1)
    
    # Normalize images
    x_train = x_train / 255.
    x_val = x_val / 255.
    x_test = x_test / 255.
    
    # Create data dictionary to contain train,val,test sets
    data_dict = {
        "x_train":x_train,
        "y_train":y_train,
        "x_val":x_val,
        "y_val":y_val,
        "x_test":x_test,
        "y_test":y_test
    }
    
    return data_dict

In [ ]:
def load_data():
    
    x_train = []
    y_train = []
    
    folder_names = ["campus_run1", "campus_run2", "run1", "run2", "run3", "second_spot1", "second_spot2"]
    
    path = "/steering_new/"
    
    # Loop folders
    for folder in folder_names:
        
        # open folder
        os.chdir(path + folder)
        
        # Loop files
        for file in sorted(glob.glob("*.jpg")):

            # Load image
            x_train.append(file)

    count = len(x_train) // 5 # 20%
    
    # Extract val set
    x_val = x_train[-count:]
    x_train = x_train[:-count]
    
    # Extract test set
    x_test = x_train[-count:]
    x_train = x_train[:count]
    
    # Load val set
    y_val = []
    for file in x_val:

        # Load image
        x_val.append(imread(file))

        # Extract label
        value = file.split('_')[-1].replace('.jpg','') # Remove the idx from filepath
        y_val.append(radians_to_degrees(float(value)))
        
    # Loat test set
    y_test = []
    for file in x_test:

        # Load image
        x_test.append(imread(file))

        # Extract label
        value = file.split('_')[-1].replace('.jpg','') # Remove the idx from filepath
        y_test.append(radians_to_degrees(float(value)))
    
    # Create data dictionary to contain train,val,test sets
    data_dict = {
        "x_train":x_train,
        "y_train":y_train,
        "x_val":x_val,
        "y_val":y_val,
        "x_test":x_test,
        "y_test":y_test
    }
    
    return data_dict

In [ ]:
def init_model(data_dict, hyperparameters):
    
    base_model = InceptionV3(weights="imagenet", include_top=False, input_shape=data_dict["x_train"][1].shape)
    
    # add a global spatial average pooling layer
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    
    # add a fully-connected layer
    x = Dense(hyperparameters["fc_size"], activation=hyperparameters["fc_activation"])(x)
    
    # add a logistic layer
    predictions = Dense(1, kernel_initializer='normal')(x)
    #Dense(num_classes, activation=output_activation)(x)
    
    # train this model
    model = Model(inputs=base_model.input, outputs=predictions)
    
    # first: train only the top layers (which were randomly initialized)
    # i.e. freeze all convolutional InceptionV3 layers
    for layer in base_model.layers:
        layer.trainable = False

    # compile the model (should be done *after* setting layers to non-trainable)
    model.compile(optimizer='adam', loss=hyperparameters["loss"], metrics=hyperparameters["metrics"])
    
    return model

In [ ]:
#def main():

# Hyperparameters
hyperparameters = {
    "batchsize" : 32,
    "fc_size" : 1024,
    "fc_activation" : 'relu',
    "epoch_finetune" : 50,
    "epoch_transfer" : 50,
    "loss" : "mean_squared_error",
    "metrics" : None,#["accuracy"]
    "monitor" : 'val_loss'
}

# # Load steering images
data_dict = load_data()

# # Create model
model = init_model(data_dict, hyperparameters)

m = data_dict["x_train"].shape[0]

# Train model
#train_model(data_dict, hyperparameters)

#    return

In [ ]:
# def train_model(data_dict, hyperparameters):
    
top_weights_path = "/output/top_weights.h5"

# Early Stopping and Model Checkpoint callbacks
callbacks_list = [
    ModelCheckpoint(top_weights_path, monitor=hyperparameters["monitor"], verbose=1, save_best_only=True),
    EarlyStopping(monitor=hyperparameters["monitor"], patience=5, verbose=0)
]

# train the model on the new data for a few epochs
history1 = model.fit_generator(generator(data_dict, hyperparameters["batchsize"]), steps_per_epoch= m / hyperparameters["batchsize"], 
                    epochs=hyperparameters["epoch_finetune"], callbacks=callbacks_list, validation_data=(data_dict["x_val"], data_dict["y_val"]))

model.load_weights(top_weights_path)

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:249]:
    layer.trainable = False
for layer in model.layers[249:]:
    layer.trainable = True

# Early Stopping and Model Checkpoint callbacks
final_weights_path = "/output/final_weights.h5"
callbacks_list = [
    ModelCheckpoint(final_weights_path, monitor=hyperparameters["monitor"], verbose=1, save_best_only=True),
    EarlyStopping(monitor=hyperparameters["monitor"], patience=5, verbose=0)
]

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss=hyperparameters["loss"], metrics=hyperparameters["metrics"])

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
history2 = model.fit_generator(generator(data_dict,hyperparameters["batchsize"]), steps_per_epoch= m / hyperparameters["batchsize"], 
                    epochs=hyperparameters["epoch_transfer"], callbacks=callbacks_list, validation_data=(data_dict["x_val"], data_dict["y_val"]))

    # Save the transfer learning and fine tuning history
    #save_training_history(history1, history2)
    
#     return

In [ ]:
final_weights_path = "/output/final_weights.h5"

model.load_weights(final_weights_path)

preds = model.evaluate(data_dict["x_test"], data_dict["y_test"])

print(preds)

In [ ]:
plt.plot(history1.history['val_loss'])

In [ ]:
plt.plot(history2.history['val_loss'])